# 8장. 생성 모델을 위한 딥러닝(Deep Learning for generative models)

* 이 장에서 다룰 핵심 내용
    * LSTM을 사용할 텍스트 생성
    * 딥드림 구현
    * 뉴럴 스타일 트랜스퍼 구현
    * 변이형 오토인코더
    * 적대적 생성 네트워크 이해

* 2015년 여름 구글의 딥드림 알고리즘이 이미지를 강아지 눈과 환상적인 인공물이 뒤섞인 몽환적인 그림으로 바꾸어 많은 사람들의 흥미를 끌었습니다. 2016년에는 프리즈마(Prizma) 애플리케이션이 사람들의 사진을 다양한 스타일의 그림으로 바꾸어 주었습니다. 2016년 여름 실험적인 단편 영화 <**썬스프링**(Sunspring)>은 LSTM 알고리즘이 쓴 완전한 대화가 포함된 각본을 사용하여 만들어졌습니다. 아마도 신경망이 시험 삼아 작곡한 음악도 최근에 들어 보았을 것입니다.
* 사실 지금까지 본 AI 예술 작품의 품질은 낮습니다. AI가 실제 작가나 화가, 작곡가와 견줄 수준은 못 됩니다. 사람을 대신한다는 것은 늘 핵심을 벗어난 생각입니다.인공 지능이 사람의 지능을 어떤 다른 것으로 대체하지 않습니다. 인공 지능은 우리 생활과 일에 지능을 더합니다. 이 지능은 다른 종류의 지능입니다. 여러 분야에서 특히 예술에서는 AI가 사람의 능력을 증가시키는 도구로 사용될 것입니다. 즉 인공적인 지능이 아니라 **확장된 지능**(augmented intelligence)입니다.
* 예술 창작의 대부분은 간단한 패턴 인식과 기술로 만들어집니다. 많은 사람은 이런 과정에 관심이 없거나 심지어 불필요하다고 생각하기도 합니다. 여기에 AI가 필요합니다. 사람의 지각, 언어, 예술 작품은 모두 통계적 구조를 가집니다. 딥러닝 알고리즘은 이 구조를 학습하는 데 뛰어납니다. 머신 러닝 모델은 이미지, 음악, 글의 통계적 **잠재 공간**(latent space)을 학습할 수 있습니다. 그다음 이 공간에서 샘플을 뽑아 새로운 예술 작품을 만들 수 있습니다. 이 작품은 모델이 훈련 데이터에서 본 것과 비슷한 특징을 가질 것입니다. 당연히 이런 샘플링 자체가 예술 창작이라고 보기는 어렵습니다. 사실 수학 연산에 불과합니다. 알고리즘은 사람의 삶, 감정, 세성에 대한 경험이 없습니다. 사람과 매우 다른 경험에서 배웁니다. 이는 단지 제3자의 입장에서 알고리즘이 만든 것의 의미를 해석한 것입니다. 숙련된 예술가가 사용하면 알고리즘 창작은 의미 있고 아름다운 것으로 바뀔 수 있습니다. 잠재 공간 샘플링은 예술가의 능력을 높이는 붓이 될 수 있습니다. 또 창작 가능성을 늘리며 상상의 공간을 확장시킵니다. 이외에도 기교와 훈련이 필요하지 않기 때문에 예술 창작을 더욱 자유롭게 만들어 줍니다. 기교에서 예술을 분리하여 순수한 표현을 만드는 새로운 도구입니다.
* 전자 음악과 컴퓨터 음악의 선구자인 이안니스 크세나키스(Iannis Xenakis)는 1960년대에 자동 작곡 애플리케이션에 대해 동일한 아이디어를 아름답게 표현했습니다.
    * 작곡가가 지루한 작업에서 벗어나면 새로운 음악 형식에 관한 일반적인 문제에 집중하고 입력 데이터를 수정하면서 이 형식의 구석구석을 탐험할 수 있습니다. 예를 들어 독주에서 실내 악단, 대형 오케스트라까지 모든 악기의 조합을 테스트할 수 있습니다. 컴퓨터를 활용하는 작곡가는 일종의 파일럿입니다. 출발 버튼을 누르고 좌표를 설정하고 소리의 우주를 항해하는 우주선을 조정합니다. 이전에는 머나먼 꿈처럼 생각했던 음악의 별자리와 은하 사이를 여행하게 됩니다.
* 이 장에서 예술 창작에 딥러닝이 어떻게 쓰일 수 있는지 다양한 각도에서 살펴보겠습니다. 시퀀스 데이터 생성(글을 쓰거나 작곡할 수 있습니다)과 딥드림, 변이형 오토인코더, 적대적 생성 네트워크(generative adversarial network)를 사용한 이미지 생성을 알아보겠습니다. 컴퓨터가 새로운 꿈을 꾸도록 만들어 보죠. 어쩌면 우리도 꿈을 꾸게 될지 모릅니다. 기술과 예술의 접점에 놓인 놀라운 가능성에 관한 꿈입니다.

## 8.1 LSTM으로 텍스트 생성하기

* 이 절에서 순환 신경망으로 시퀀스 데이터를 생성하는 방법을 살펴보겠습니다. 텍스트 생성을 예로 들지만 동일한 기법으로 어떤 종류의 시퀀스 데이터도 생성할 수 있습니다. 음표의 시퀀스에 적용하여 새로운 음악을 만들거나 연속된 붓질 시퀀스(예를 들어 화가가 아이패드에 그림을 그리는 과정을 기록한 것)에 적용하여 한 획 한 획 그림을 그릴 수 있습니다.
* 시퀀스 데이터 생성이 예술 콘텐츠에 국한되지 않습니다. 음성 합성과 챗봇의 대화 기능에 성공적으로 적용되었습니다. 구글이 2016년에 공개한 스마트 답장(Smart Reply)도 비슷한 기술을 사용합니다. 짧은 문장을 자동으로 생성하여 이메일이나 문자 메시지로 답장을 보낼 수 있습니다.

### 8.1.1 생성 RNN(Recurrent Neural Network 순환 신경망)의 간단한 역사


* 2014년 후반 머신 러닝 공동체에서도 소수의 사람들만이 LSTM이란 용어를 알았습니다. 순환 네트워크(RNN)를 사용하여 시퀀스 데이터(Sequence data)를 성공적으로 생성한 애플리케이션은 2016년이 되어서야 주류가 되기 시작했습니다.하지만 이 기술은 1997년 LSTM 알고리즘이 개발된 이래 역사가 꽤 오래 되었습니다. 이 알고리즘은 초기에 글자를 하나씩 생성하는 데 사용되었습니다.
* 2002년 더글라스 에크(Douglas Eck)는 스위스의 슈미드후버(Schmidhuber)의 연구실에서 LSTM을 음악 생성에 처음 적용하여 가능성 있는 결과를 얻었습니다. 에크는 현재 구글 브레인의 연구원으로, 2016년에는 마젠타(Magenta)란 새로운 연구 그룹을 시작했습니다. 여기에서는 최신 딥러닝 기술을 사용하여 멋진 음악을 만드는 것에 집중하고 있습니다. 때로는 좋은 아이디어가 실현되는데 15년이 걸리기도 합니다.
* 2000년대 후반과 2010년대 초반에 알렉스 그레이브스(Alex Graves)는 순환 네트워크를 사용하여 시퀀스 데이터를 생성하는 데 아주 중요한 선구적인 일을 했습니다. 특히 2013년에 펜 위치를 기록한 시계열 데이터를 사용하여 순환 네트워크와 완전 연결 네트워크를 혼합한 네트워크로 사람이 쓴 것 같은 손글씨를 생성했으며, 이 작업이 전환점이 되었습니다. 때맞추어 등장한 특별한 이 신경망 애플리케이션은 꿈을 꾸는 컴퓨터를 상상하게 만들었고, 필자가 케라스를 개발할 때 많은 영감을 주었습니다. 그레이브스는 논문 사전 출판 서버인 아카이브(arXiv)에 업로드한 2013년 레이텍(LaTex) 파일에 비슷한 주석을 남겼습니다. "시퀀스 데이터를 생성하는 것은 컴퓨터가 꿈을 꾸게 하는 것입니다." 몇 년이 지난 지금 우리는 이런 애플리케이션을 당연하게 받아들입니다. 그 당시에는 그레이브스의 실험을 보고 가능성만으로 대단한 것이라고 생각하기는 어려웠습니다.  
* 그 이후 순환 신경망은 음악 생성, 대화 생성, 이미지 생성, 음성 합성, 분자 설계(molecule design)에 성공적으로 사용되었습니다. 심지어 실제 배우가 캐스팅된 영화의 대본을 만드는 데도 사용되었습니다.

### 8.1.2 시퀀스 데이터를 어떻게 생성할까?
* 딥러닝에서 시퀀스 데이터를 생성하는 일반적인 방법은 이전 토큰을 입력으로 사용해서 시퀀스의 다음 1개 또는 몇 개의 토큰을 (RNN이나 컨브넷(convnet)으로) 예측하는 것입니다. 예를 들어 "the cat is on ma"란 입력이 주어지면 다음 글자인 타깃 "t"를 예측하도록 네트워크를 훈련합니다. 텍스트 데이터를 다룰 때 토큰은 보통 단어 또는 글자입니다. 이전 토큰들이 주어졌을 때 다음 토큰의 확률을 모델링할 수 있는 네트워크를 **언어 모델**(language model)이라고 부릅니다. 언어 모델은 언어의 통계적 구조인 잠재 공간을 탐색합니다.
* 언어 모델을 훈련하고 나면 이 모델에서 샘플링을 할 수 있습니다(새로운 시퀀스를 생성합니다). 초기 텍스트 문자열을 주입하고(**조건 데이터**(conditioning data)라고 부릅니다) 새로운 글자나 단어를 생성합니다(한 번에 여러 개의 토큰을 생성할 수도 있습니다). 생성된 출력은 다시 입력 데이터로 추가됩니다. 이 과정을 여러 번 반복합니다. 이런 반복을 통해 모델이 훈련한 데이터 구조가 반영된 임의의 길이를 가진 시퀀스를 생성할 수 있습니다. 사람이 쓴 문장과 거의 비슷한 시퀀스를 만듭니다. 이 절의 예제는 LSTM층을 사용합니다. 텍스트 말뭉치(corpus)에서 N개의 글자로 이루어진 문자열을 추출하여 주입하고 N + 1번째 글자를 예측하도록 훈련합니다. 모델의 출력은 출력 가능한 모든 글자에 해당하는 소프트맥스(softmax) 값입니다. 즉 다음 글자의 확률 분포입니다. 이 LSTM을 **글자 수준의 신경망 언어 모델**(character-level neural language model)이라고 부릅니다.

### 8.1.3 샘플링 전략의 중요성
* 텍스트를 생성할 때 다음 글자를 선택하는 방법이 아주 중요합니다. 단순한 방법은 항상 가장 높은 확률을 가진 글자를 선택하는 **탐욕적 샘플링**(greedy sampling)입니다. 이 방법은 반복적이고 예상 가능한 문자열을 만들기 때문에 논리적인 언어처럼 보이지 않습니다. 좀 더 흥미로운 방법을 사용하면 놀라운 선택이 만들어집니다. 다음 글자의 확률 분포에서 샘플링하는 과정에 무작위성을 주입하는 방법입니다. 이를 **확률적 샘플링**(stochastic sampling)이라고 부릅니다(머신 러닝에서 확률적(stochastic)이란 뜻은 무작위(random)하다는 뜻입니다). 이런 방식을 사용할 경우 'e'가 다음 글자가 될 확률이 0.3 이라면, 모델이 30%정도는 이 글자를 선택합니다. 탐욕적 샘플링을 확률적 샘플링으로 설명할 수도 있습니다. 한 글자의 확률이 1이고 나머지 글자는 모두 0인 확률 분포를 가지는 경우입니다.
* 모델의 소프트맥스 출력은 확률적 샘플링에 사용하기 좋습니다. 이따금 샘플링될 것 같지 않은 글자를 샘플링합니다. 훈련 데이터에는 없지만 실제 같은 새로운 단어를 만들어 재미있고 창의적으로 보이는 문장을 생성합니다. 이 전략에는 한 가지 문제가 있습니다. 샘플링 과정에서 무작위성의 양을 조절할 방법이 없습니다.
* 왜 무작위성이 크거나 작아야 할까요? 극단적인 경우를 생각해 보죠. 균등 확률 분포에서 다음 글자를 추출하는 완전한 무작위 샘플링이 있습니다. 모든 글자의 확률은 같습니다. 이 구조는 무작위성이 최대입니다. 다른 말로 하면 이 확률 분포는 최대의 엔트로피를 가집니다. 당연하게 흥미로운 것들을 생산하지 못합니다. 반대의 경우 탐욕적 샘플링은 무작위성이 없기 때문에 흥미로운 것을 전혀 만들지 못합니다. 탐욕적 샘플링의 확률 분포는 최소의 엔트로피를 가집니다. 모델의 소프트맥스 출력인 '실제' 확률 분포에서 샘플링하는 것은 이 두 극단의 중간에 위치해 있습니다. 중간 지점에는 시도해 볼 만한 더 높거나 낮은 엔트로피가 많습니다. 작은 엔트로피는 예상 가능한 구조를 가진 시퀀스를 생성합니다(더 실제처럼 보입니다). 반면에 높은 엔트로피는 놀랍고 창의적인 시퀀스를 만듭니다. 생성 모델에서 샘플링을 할 때 생성 과정에 무작위성의 양을 바꾸어 시도해 보는 것이 좋습니다. 흥미는 매우 주관적이므로 최적의 엔트로피 값을 미리 알 수 없기 때문입니다. 얼마나 흥미로운 데이터를 생성할 것인지는 결국 사람이 판단해야 합니다.
* 샘플링 과정에서 확률의 양을 조절하기 위해 **소프트맥스 온도**(softmax temperature)라는 파라미터를 사용합니다. 이 파라미터는 샘플링에 사용되는 확률 분포의 엔트로피를 나타냅니다. 얼마나 놀라운 또는 예상되는 글자를 선택할 지 결정합니다. temperature 값이 주어지면 다음과 같이 가중치를 적용하여 (모델의 소프트맥스 출력인) 원본 확률 분포에서 새로운 확률 분포를 계산합니다.

In [ ]:
# 다른 온도 값을 사용하여 확률 분포의 가중치 바꾸기
# Replace the weight of a probability distribution by using different temperature values
import numpy as np

# original distribution은 전체 합이 1인 1D 넘파일 배열입니다.
# temperature는 출력 분포의 엔트로피 양을 결정합니다.
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    # 원본 분포의 가중치를 변경하여 반환합니다.
    # 이 분포의 합은 1이 아닐 수 있으므로 새로운 분포의 합으로 나눕니다.
    return distribution / np.sum(distribution)

* 높은 온도는 엔트로피가 높은 샘플링 분포를 만들어 더 놀랍고 생소한 데이터를 생성합니다. 반면에 낮은 온도는 무작위성이 낮기 때문에 예상할 수 있는 데이터를 생산합니다.

★ LSTM으로 텍스트 생성하기(Text generation with LSTM)

### 8.1.4 글자 수준의 LSTM 텍스트 생성 모델 구현
* 이런 아이디어를 케라스로 구현해 보죠. 먼저 언어 모델을 학습하기 위해 많은 텍스트 데이터가 필요합니다. 위키피디아나 반지의 제왕처럼 아주 큰 텍스트 파일이나 텍스트 파일의 묶음을 사용할 수 있습니다. 이 예에서는 19세기 후반 독일의 철학자 니체(Nietzsche)의 글을 사용하겠습니다(영어로 번역된 글입니다). 학습할 언어 모델은 일반적인 영어 모델이 아니라 니체의 문체와 특정 주제를 따르는 모델일 것입니다.

#### 데이터 전처리
* 먼저 말뭉치를 다운로드하고 소문자로 바꿉니다.

In [3]:
# 원본 텍스트 파일을 내려받아 파싱하기
# Download and parse the original text file
import tensorflow.keras
import numpy as np

path = tensorflow.keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('말뭉치 크기:', len(text))

606208/600901 [==============================] - 1s 2us/step
말뭉치 크기: 600893


In [4]:
type(text)

str

* 그 다음 `maxlen` 길이를 가진 시퀀스를 중복하여 추출합니다. 추출된 시퀀스를 원-핫 인코딩으로 변환하고 크기가 `(sequences, maxlen, unique_characters)`인 3D 넘파이 배열 `x`로 합칩니다. 동시에 훈련 샘플에 상응하는 타깃을 담은 배열 `y`를 준비합니다. 타깃은 추출된 시퀀스 다음에 오는 원-핫 인코딩된 글자입니다.

In [5]:
# 글자 시퀀스 벡터화하기
# Vectorize the character sequence

# 60개 글자로 된 시퀀스를 추출합니다.
maxlen = 60

# 세 글자씩 건너 뛰면서 새로운 시퀀스를 샘플링합니다.
step = 3

# 추출한 시퀀스를 담을 리스트
sentences = []

# 타깃(시퀀스 다음 글자)을 담을 리스트
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('시퀀스 개수:', len(sentences))

# 말뭉치에서 고유한 글자를 담은 리스트
chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿉니다.
print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 200278
고유한 글자: 58
벡터화...


#### 네트워크 구성
* 이 네트워크는 하나의 `LSTM` 층과 그 뒤에 `Dense` 분류기가 뒤따릅니다. 분류기는 가능한 모든 글자에 대한 소프트맥스 출력을 만듭니다. 순환 신경망이 시퀀스 데이터를 생성하는 유일한 방법은 아닙니다. 최근에는 1D 컨브넷도 이런 작업에 아주 잘 들어 맞는다는 것이 밝혀졌습니다.

In [8]:
# 다음 글자를 예측하기 위한 단일 LSTM 모델 만들기
# create single LSTM model to predict the next character
from tensorflow.keras import layers

model = tensorflow.keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

* 타깃이 원-핫 인코딩되어 있기 때문에 모델을 훈련하기 위해 `categorical_crossentropy` 손실을 사용합니다.

In [9]:
# 모델 컴파일 설정하기
# set up model compile
optimizer = tensorflow.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

c:\users\wnstj\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


#### 언어 모델 훈련과 샘플링(Training language modle and sampling)

* 훈련된 모델과 시드로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있습니다.

    1.	지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑습니다.
    2.	특정 온도로 이 확률 분포의 가중치를 조정합니다.
    3.	가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다.
    4.	새로운 글자를 생성된 텍스트의 끝에 추가합니다.

* 다음 코드는 모델에서 나온 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추출합니다(샘플링 함수입니다).

In [10]:
# 모델의 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수 구현하기
# implement Function to sample new characters with given the model's prediction
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

* 마지막으로 다음 반복문은 반복적으로 훈련하고 텍스트를 생성합니다. 에포크마다 학습이 끝난 후 여러가지 온도를 사용해 텍스트를 생성합니다. 이렇게 하면 모델이 수렴하면서 생성된 텍스트가 어떻게 진화하는지 볼 수 있습니다. 온도가 샘플링 전략에 미치는 영향도 보여 줍니다.

In [ ]:
# 텍스트 생성 루프 만들기
# create text generation loop
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

# 60 에포크 동안 모델을 훈련합니다
for epoch in range(1, 60):
    print('에포크', epoch)
    # 데이터에서 한 번만 반복해서 모델을 학습합니다
    model.fit(x, y, batch_size=128, epochs=1)

    # 무작위로 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 시도합니다
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 400개의 글자를 생성합니다
        for i in range(400):
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링합니다
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
1565/1565 [==============================] - 360s 228ms/step - loss: 1.9605
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the more the compertal and there is the more something the is the completed the great the more there is the completely and the the serious and all the been and the one stand and the the the in the same there is been the more been of the been the inderitions of the good the experience, there is a proterst the as the been the something the experience, in the sears and the sears of the preations of
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the fares of the beer one the as it is it as there is for a perhaps it is the conderstion of a beenst there in the become as the great the preas the sasters there is has one many in the belistations, and to been so there is the mame to the sething there is a perhaps of the communice and st

* 여기서 볼 수 있듯이 낮은 온도는 아주 반복적이고 예상되는 텍스트를 만듭니다. 하지만 국부적인 구조는 매우 실제와 같습니다. 특히 모든 단어(단어는 글자의 지역 패턴으로 이루어집니다)가 실제 영어 단어입니다. 높은 온도에서 생성된 텍스트는 아주 흥미롭고 놀라우며 창의적이기도 합니다. 이따금 꽤 그럴싸하게 보이는 완전히 새로운 단어를 창조합니다(‘begarmed’와 ‘isharent’ 같은 단어입니다). 높은 온도에서는 국부적인 구조가 무너지기 시작합니다. 대부분의 단어가 어느정도 무작위한 문자열로 보입니다. 확실히 이 네트워크에서는 텍스트 생성에 가장 좋은 온도는 0.5입니다. 항상 다양한 샘플링 전략으로 실험해 봐야합니다! 학습된 구조와 무작위성 사이에 균형을 잘 맞추면 흥미로운 것을 만들 수 있습니다.

* 더 많은 데이터에서 크고 깊은 모델을 훈련하면 이것보다 훨씬 논리적이고 실제와 같은 텍스트 샘플을 생성할 수 있습니다. 당연히 우연이 아닌 의미 있는 텍스트가 생성된다고 기대하지 마세요. 글자를 연속해서 나열하기 위한 통계 모델에서 데이터를 샘플링한 것뿐입니다. 언어는 의사소통의 수단입니다. 의사소통이 의미하는 것과 의사소통이 인코딩된 메시지의 통계 구조 사이는 차이가 있습니다. 이 차이를 검증하기 위해 다음과 같은 사고 실험을 해보죠. 컴퓨터가 대부분의 디지털 통신에서 하는 것처럼 사람의 언어가 의사소통을 압축하는데 더 뛰어나다면 어떨까요? 언어의 의미가 줄진 않지만 고유한 통계 구조가 사라질 것입니다. 이는 방금과 같은 언어 모델을 학습하는 것을 불가능하게 만듭니다.

### 8.1.5 정리

* 이전의 토큰이 주어지면 다음 토큰(들)을 예측하는 모델을 훈련하여 시퀀스 데이터를 생성할 수 있습니다.
* 텍스트의 경우 이런 모델을 **언어 모델**이라 부릅니다. 단어 또는 글자 단위 모두 가능합니다.
* 다음 토큰을 샘플링할 때 모델이 만든 출력에 집중하는 것과 무작위성을 주입하는 것 사이에 균형을 맞추어야 합니다.
* 이를 위해 소프트맥스 온도 개념을 사용합니다. 항상 다양한 온도를 실험해서 적절한 값을 찾습니다.